In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [2]:
print_to_excel = True
excel_tofolder_on_Z = True

In [3]:
excel_data = pd.read_excel('report_invest_30.06.2024.xlsx', sheet_name='snapshot', header=10)

In [4]:
excel_data_work = excel_data
# to usd
excel_data_work['Заказ на поставку'] = excel_data_work['Заказ на поставку'].astype('float').fillna(0)
excel_data_work = add_in_currency_column(excel_data_work,'USD','ПВК|Тип прогноз. данных',False,'Заказ на поставку','2024-05-30')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [5]:
excel_data_work['holding'] = merge_SalesUnits(excel_data_work,'Балансовая единица',merge_col='holding') # merge holding
# create version columns
new_cols = ['Версия_045', 'Версия_075', 'Версия_035', 'Версия_085']
excel_data_work[new_cols] = None
for i in excel_data_work.index:
    excel_data_work.loc[i, [x for x in new_cols if str(excel_data_work.loc[i,'Версия']) in x][0]] = excel_data_work.loc[i,'Заказ на поставку_in_USD']
excel_data_version_pivot = excel_data_work.pivot_table(index=['Балансовая единица','Завод',
       'ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта',
       'Проект','Вид затрат','КалендГод/Месяц', 'Календарный год',
       'Календарный месяц','holding'], values=new_cols, aggfunc=sum).reset_index().sort_values(['Балансовая единица','Контрагент'])
# compaire values

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK' 'SUEK' 'SUEK' ... 'SUEK' 'SUEK' 'SUEK']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_8388\3713946399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [6]:
excel_data_work['holding'].unique()

array(['EUROCHEM', 'SUEK', 'External'], dtype=object)

In [16]:
for version_col in new_cols:
    excel_data_version_pivot.loc[excel_data_version_pivot[version_col]==0,version_col]=np.NaN
excel_data_version_pivot = excel_data_version_pivot[~excel_data_version_pivot.Версия_045.isna()]
excel_data_version_pivot = concat_columns(excel_data_version_pivot,['Балансовая единица','Завод','ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта','Проект','Вид затрат','holding'])
excel_data_contract_pivot = excel_data_version_pivot.pivot_table(index=['concat_columns','holding'],values=new_cols,aggfunc=['count',sum]).reset_index()
# Если сумма платежей (значение поля " версия " = 075) больше суммы освоения (значение поля " версия " = 035 для БЕ из holding = SUEK; либо " версия " = 085 для БЕ из holding = Eurochem), то берем сумму освоения (035 или 085) и равномерно распределяем по срокам, соответствующим будущим платежам (075), начиная с самого дальнего. Т.к. объём освоения меньше, чем платежей, то таким образом будет "заполнена" только какая-то часть платежей. В итоге в unifiedOCP надо забрать платежи c НДС (045), взятые в той же пропорции, в которой были заполнены платежи без НДС (075)
excel_data_contract_pivot_3th = excel_data_contract_pivot[((excel_data_contract_pivot.holding=='SUEK')\
                                                           &(excel_data_contract_pivot['sum']['Версия_035']<excel_data_contract_pivot['sum']['Версия_075']))\
                                                            |((excel_data_contract_pivot.holding=='EUROCHEM')&\
                                                              (excel_data_contract_pivot['sum']['Версия_085']<excel_data_contract_pivot['sum']['Версия_075']))]
# 035>1 и не равна 075 (тогда процент равен 1)
df_3th_full = pd.DataFrame()
version_col = ''
for hold in ['SUEK','EUROCHEM']:
    if hold == 'SUEK':
        version = 'Версия_075'
    if hold == 'EUROCHEM':
        version = 'Версия_085'

    excel_data_contract_pivot_3th = excel_data_contract_pivot_3th[((excel_data_contract_pivot_3th['count']['Версия_035']>1)&(excel_data_contract_pivot_3th['count']['Версия_035']!=excel_data_contract_pivot_3th['count'][version]))&(excel_data_contract_pivot_3th.holding==hold)]

    excel_data_contract_pivot_3th['number_of_45_rows_need_to_get'] = (excel_data_contract_pivot_3th['count']['Версия_035']/excel_data_contract_pivot_3th['count'][version]) * excel_data_contract_pivot_3th['count']['Версия_045']

    excel_data_contract_pivot_3th.loc[excel_data_contract_pivot_3th['count']['Версия_045']<=excel_data_contract_pivot_3th.number_of_45_rows_need_to_get,'number_of_45_rows_need_to_get'] = np.NaN

    excel_data_contract_pivot_3th = excel_data_contract_pivot_3th[~excel_data_contract_pivot_3th.number_of_45_rows_need_to_get.isna()]
    if len(excel_data_contract_pivot_3th) == 0:
        continue
    
    excel_data_contract_pivot_3th['index_045_from'] = excel_data_contract_pivot_3th['count']['Версия_045'] - excel_data_contract_pivot_3th.number_of_45_rows_need_to_get
    # create dict
    dict_for_3th = {x:y for x,y in zip(excel_data_contract_pivot_3th.concat_columns.values.tolist(), excel_data_contract_pivot_3th.index_045_from.values.tolist())}
    # full data
    exit_df = pd.DataFrame()
    excel_data_version_pivot['amount'] = None
    for contract in list(dict_for_3th.keys()):
        contract_df = excel_data_version_pivot[excel_data_version_pivot.concat_columns==contract].reset_index(drop=True)
        contract_df.loc[dict_for_3th[contract]:,'amount'] = contract_df.loc[dict_for_3th[contract]:,'Версия_045']
        # excel_data_version_pivot[(excel_data_version_pivot.concat_columns==contract)&(~contract_df.Версия_045.isna())] = contract_df
        exit_df = pd.concat([exit_df,contract_df],axis=0)
    df_3th = exit_df.pivot_table(index=['Балансовая единица','Завод','ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта','Проект','Вид затрат','holding'],values=new_cols+['amount'],aggfunc=sum).reset_index()
    df_3th_full = pd.concat([df_3th_full,df_3th])

for_conditions_pivot = excel_data_version_pivot.pivot_table(index=['Балансовая единица','Завод','ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта','Проект','Вид затрат','holding'],values=new_cols+['amount'],aggfunc=sum).reset_index()
# Если сумма платежей (значение поля " версия " = 075) равна сумме освоения (значение поля " версия " = 035 для БЕ из holding = SUEK; либо " версия " = 085 для БЕ из holding = Eurochem), то берем платежи с НДС (значение поля " версия " = 045)
# Если сумма платежей (значение поля " версия " = 075) меньше суммы освоения (значение поля " версия " = 035 для БЕ из holding = SUEK; либо " версия " = 085 для БЕ из holding = Eurochem), то берем платежи с НДС (значение поля " версия " = 045)
df_1_2th = for_conditions_pivot[((for_conditions_pivot.holding=='SUEK')\
                                &(for_conditions_pivot['Версия_035']>=for_conditions_pivot['Версия_075']))\
                                    |((for_conditions_pivot.holding=='EUROCHEM')\
                                &(for_conditions_pivot['Версия_035']>=for_conditions_pivot['Версия_085']))]
df_1_2th['amount'] = df_1_2th['Версия_045']

df_to_print = pd.concat([df_1_2th,df_3th],axis=0)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_8388\1992252099.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  excel_data_contract_pivot = excel_data_version_pivot.pivot_table(index=['concat_columns','holding'],values=new_cols,aggfunc=['count',sum]).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_8388\1992252099.py:41: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_3th = exit_df.pivot_table(index=['Балансовая единица','Завод','ПЕ','СПП-элемент','Контрагент','Вид проекта','Профиль проекта','Проект','Вид затрат','holding'],values=new_cols+['amount'],aggfunc=sum).reset_index()
C:\Users\Kli

In [30]:
list(filter(lambda x: 'Ex' not in x ,df_to_print.holding.unique().tolist()))

['SUEK', 'EUROCHEM']

In [31]:
### to excel
hold = ''
hold_list = list(filter(lambda x: 'Ex' not in x ,df_to_print.holding.unique().tolist()))
for hold in hold_list:
    data = 0
    if print_to_excel == True:
        Output_file = f'{str(date.today())}_Invest_nv_{hold}.xlsx'
        if excel_tofolder_on_Z == True:
            Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
            Output_file = Output_path + Output_file
        data = df_to_print[df_to_print.holding==hold]
        data.to_excel(Output_file, sheet_name='Invest', index=False)

In [19]:
assert 2==1

AssertionError: 

In [ ]:
# compaire with old BD
old_BD_data_SUEK = pd.read_excel('2024-06-13_SUEK_Invest.xlsx',header=3).drop(0,axis=0) 
old_BD_data_SUEK_pivot = old_BD_data_SUEK.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
old_BD_data_EURO = pd.read_excel('2024-06-13_EUROCHEM_Invest.xlsx',header=3).drop(0,axis=0) 
old_BD_data_EURO_pivot = old_BD_data_EURO.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
old_BD_data_SUEK_pivot

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\4027979718.py:3: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  old_BD_data_SUEK_pivot = old_BD_data_SUEK.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Законтрактовано.2':'Сумма_итого_законтрактровнано'})
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\4027979718.py:5: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  old_BD_data_EURO_pivot = old_BD_data_EURO.pivot_table(index='Unnamed: 5',values='Законтрактовано.2',aggfunc=sum).reset_index().rename(columns={'Unnamed: 5':'БЕ','Зак

,БЕ,Сумма_итого_законтрактровнано
0,1100,156069493.09
1,1300,72101.36
2,1600,19216.02
3,1I00,104629.4
4,1M00,1584353.68
...,...,...
76,Y100,155526600.61
77,Y300,18343444.65
78,Y500,26009.73
79,Y600,0


In [ ]:
compaire_to_print = df_to_print.pivot_table(index='Балансовая единица',aggfunc=sum,values='amount').reset_index().merge(old_BD_data_SUEK_pivot,how='outer',left_on='Балансовая единица',right_on='БЕ').sort_values('amount',ascending=False)
compaire_to_print = compaire_to_print.fillna(0)
compaire_to_print['is_equal'] = compaire_to_print.amount.astype(int)==compaire_to_print['Сумма_итого_законтрактровнано'].astype(int)
compaire_to_print = compaire_to_print[compaire_to_print.Сумма_итого_законтрактровнано!=0]
if print_to_excel==True:
    Output_file = f'{str(date.today())}_Invest_from_report_invest_30.05.2024.xlsx'
    compaire_to_print.to_excel(Output_file,sheet_name='compaire',index=False)
    new_list(df_to_print,Output_file,'data')

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\307249084.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  compaire_to_print = df_to_print.pivot_table(index='Балансовая единица',aggfunc=sum,values='amount').reset_index().merge(old_BD_data_SUEK_pivot,how='outer',left_on='Балансовая единица',right_on='БЕ').sort_values('amount',ascending=False)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_22624\307249084.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  compaire_to_print = compaire_to_print.fillna(0)


In [ ]:
new_list(excel_data_version_pivot[(excel_data_version_pivot.Версия_045==0)&(excel_data_version_pivot.Версия_075!=0)],Output_file,'errors')